In [285]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time



client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [286]:
program_id = PublicKey("7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz")
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")


In [287]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [288]:
accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

request_units = CStruct(
    "units" / U32,
    "additional_fee" / U32   
)

def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [289]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
client.request_airdrop(payer_keypair.public_key, 2*10**9)
base58.b58encode(payer_keypair.secret_key)

b'mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW'

In [290]:
dsol_mint = PublicKey("HSYdApQF2FKVNnX6FQ1ySYstGLe5Gp8NseHVFavWye66")
governor_mint = PublicKey("5TvLJ3Xd9LbB4q97mmYWcJqjKEpqPQE7ZEgQc5M9xt1v")
payer_bump = 2

payer_governor_token_address = instructions.get_associated_token_address(payer_keypair.public_key, governor_mint)

table_mint_keypair = Keypair.generate()

payer_table_associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, table_mint_keypair.public_key)

(payer_table_data_pda, payer_table_data_pda_bump) = PublicKey.find_program_address([b"table_data_pda", bytes(payer_keypair.public_key), payer_bump.to_bytes(4,"big")], program_id);

(payer_table_metadata_pda, payer_table_metadata_pda_bump) = PublicKey.find_program_address([b"metadata", bytes(metadata_program_pubkey), bytes(table_mint_keypair.public_key)], metadata_program_pubkey) 

(table_sales_pda, table_sales_nonce) = PublicKey.find_program_address([b'table_sales_pda'], program_id)

payer_dsol_token_account = instructions.get_associated_token_address(payer_keypair.public_key, dsol_mint)

(table_mint_authority, nonce) = PublicKey.find_program_address([b"table_mint_authority_pda"], program_id)
payer_table_data_pda

4uq6eSCtD39xgSZJrE4oKJhp6iLr6jN3aN3cEFiq9iSP

In [291]:
seed = bytes([25,215,212,154,213,198,45,123]*4)
vault_keypair = Keypair.from_seed(seed)
# vault_keypair = Keypair.generate()
# client.request_airdrop(vault_keypair.public_key, 2*10**9)
# time.sleep(5)
vault_dsol_token_account = instructions.get_associated_token_address(vault_keypair.public_key, dsol_mint)

In [292]:

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
payer_dsol_token_account_meta = AccountMeta(payer_dsol_token_account, False, True)
payer_governor_token_account_meta = AccountMeta(payer_governor_token_address, False, True)
# payer_table_associated_account_meta = AccountMeta(payer_table_associated_token_address, False, True)
payer_table_data_pda_meta = AccountMeta(payer_table_data_pda, False, True)
payer_table_metadata_meta = AccountMeta(payer_table_metadata_pda, False, True)
vault_account_meta = AccountMeta(vault_keypair.public_key, False, True)
vault_dsol_token_account_meta = AccountMeta(vault_dsol_token_account, False, True)
table_mint_account_meta = AccountMeta(table_mint_keypair.public_key, True, True)
table_mint_authority_meta = AccountMeta(table_mint_authority, False, False)
table_sales_pda_meta = AccountMeta(table_sales_pda, False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)
metadata_meta = AccountMeta(metadata_program_pubkey, False, False)
dsol_mint_meta = AccountMeta(dsol_mint, False, False)
compute_program_meta = AccountMeta(PublicKey("ComputeBudget111111111111111111111111111111"), False, False)
gov_account_meta = AccountMeta(governor_mint, False, False)


In [293]:
payer_table_data_pda_meta

AccountMeta(pubkey=4uq6eSCtD39xgSZJrE4oKJhp6iLr6jN3aN3cEFiq9iSP, is_signer=False, is_writable=True)

In [294]:


# print(dsol_mint)
# print(vault_keypair.public_key)
# print(vault_dsol_token_account)
# print(payer_keypair.public_key)
# print(payer_dsol_token_account)
# print(payer_governor_token_address)
# print(sys_program_meta)
# print(spl_program_meta)
# print(payer_table_metadata_pda)


InstructionEnum = Enum(
    "InitTable" / CStruct("governor_reward" / U32, "payer_bump" / U32),
    "SignTableMint",
    "CreateTableSalesAccount",
    "BurnNFTs" / CStruct("rarity" / U8),
    "MintTable",
    enum_name = "InstructionEnum"
)

In [295]:
# payer_bump = 0
# (table_data_pda, table_data_pda_bump) = PublicKey.find_program_address([b"table_data_pda", bytes(payer_account_meta.pubkey), payer_bump.to_bytes(4,"little")], program_id);
# payer_table_data_pda_meta = AccountMeta(payer_table_data_pda, False, True)

vault_pda, _ = PublicKey.find_program_address([b"Dsol_vault_tables"], program_id)
vault_pda_dsol_token_account_meta = AccountMeta(instructions.get_associated_token_address(vault_pda, dsol_mint), False, True)

(pending_pda, pending_nonce) = PublicKey.find_program_address([b'pending_tables'], program_id)
pending_tables_meta = AccountMeta(pending_pda, False, True)

main_vault_assoc = instructions.get_associated_token_address(PublicKey("G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z"), dsol_mint)

main_vault_meta = AccountMeta(main_vault_assoc, False, True)

gov_data_pubkey, _ = PublicKey.find_program_address([b'governor_data_pda', bytes(governor_mint)], PublicKey("GWJQHiudaVnhBmvCkRmMPeWs2rffhG2fmkXVRJHM9t5s"))
gove_data_meta = AccountMeta(gov_data_pubkey, False, True)


In [296]:
def init_table_mint():
    accounts = [
        payer_account_meta,
        payer_dsol_token_account_meta,
        payer_governor_token_account_meta,
        # payer_table_associated_account_meta,
        payer_table_data_pda_meta,
        # payer_table_metadata_meta,
        # vault_account_meta,
        vault_pda_dsol_token_account_meta,
        # table_mint_account_meta,
        # table_mint_authority_meta,
        table_sales_pda_meta,
        # sys_program_meta,
        spl_program_meta,
        # rent_account_meta,
        sysvar_clock_meta,
        pending_tables_meta,
        main_vault_meta,
        gove_data_meta,
        gov_account_meta,

        # compute_program_meta,
        spl_program_meta,
        spl_program_meta,
        sys_program_meta,
        # associated_program_meta,
        # spl_program_meta,
        # metadata_meta,
        # metadata_meta,
        # spl_program_meta,
        # spl_program_meta,
        # sys_program_meta,
    ]


    transaction = Transaction()

    # request_units = CStruct(
    # "units" / U32,
    # "additional_fee" / U32   
    # )


    # transaction.add(
    #     TransactionInstruction(
    #         [],
    #         compute_program_meta.pubkey,
    #         request_units.build({"units": 500000, "additional_fee": int(0.01*10**9)})
    #     )
    # )

    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            InstructionEnum.build(InstructionEnum.enum.InitTable(governor_reward = 500, payer_bump = payer_bump))
        )
    )
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

init_table_mint()

# time.sleep(5)

'https://explorer.solana.com/tx/41f2hkY5iYjAgtkhS77CLUFV7Gd6UBtEcxAeHtvhYyarKqdVJ7tn4RdRx4NNihUHUFLgTbJBPhdULEWyyPd2SscT?cluster=devnet'

In [297]:
program_id

7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz

In [298]:
# def mint_avatar():
#     accounts = [
#         # program_id_account_meta,
#         payer_account_meta,
#         payer_dsol_token_account_meta,
#         vault_account_meta, 
#         vault_dsol_token_account_meta,
#         mint_account_meta, 
#         rent_account_meta, 
#         associated_account_meta,
#         spl_program_meta,
#         AccountMeta(SYS_PROGRAM_ID, False, False),
#         mint_authority_meta,
#         avatar_data_pda_meta,
#         sales_pda_meta,
#         AccountMeta(metadata_pda, False, True),
#         sysvar_clock_meta,

#         spl_program_meta,
#         AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
#         spl_program_meta,
#         spl_program_meta,
#         AccountMeta(metadata_program_pubkey, False, False),
#     ]   

#     transaction = Transaction()
#     transaction.add(TransactionInstruction(
#         accounts,
#         program_id,
#         bytes([0])
#     ))
    
#     return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account, vault_keypair)['result']}?cluster=devnet"

# mint_avatar()

In [299]:

# accountSchema = CStruct(
#     "vault_total" / F32,
#     "counter" / U32,
# )

# (sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
# print(f"PDA: {sales_pda, sales_nonce}")

# getAccountInfo(client,sales_pda)

In [300]:
# accountSchema = CStruct(
#     "date_created" / U32,
#     "unlockable_date" / U32,
#     "numeration" /U32,
#     "rarity" / U8,
#     "level" / U8,
#     "xp" / U32,
# )


# (locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
# print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

# getAccountInfo(client,avatar_data_pda)

In [301]:
# time.sleep(120)

# new_accounts = [
#     mint_account_meta,
#     associated_account_meta,
#     spl_program_meta,
#     mint_authority_meta,
#     avatar_data_pda_meta,
#     sysvar_clock_meta,

#     spl_program_meta
# ]

# transaction = Transaction()
# transaction.add(TransactionInstruction(
#     new_accounts,
#     program_id,
#     bytes([1])
# ))
# f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"